In [1]:
import datasetmaker
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import openpyxl
import tifffile as tiff

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
wb = openpyxl.load_workbook(filename = r'Y:\Database\Database.xlsx')

sheet = wb.active

In [4]:
ids = [s.value for s in sheet['A'][1:]]

onlypores = [s.value for s in sheet['H'][1:]]

masks = [s.value for s in sheet['G'][1:]]

datasets1 = [s.value for s in sheet['I'][1:]]

datasets2 = [s.value for s in sheet['J'][1:]]

datasets1_lente = [s.value for s in sheet['Q'][1:]]

datasets2_lente = [s.value for s in sheet['R'][1:]]

rfs = [s.value for s in sheet['L'][1:]]

rfs_lente = [s.value for s in sheet['O'][1:]]

onlypores_lentes = [s.value for s in sheet['V'][1:]]

masks_lentes = [s.value for s in sheet['U'][1:]]

amps = [s.value for s in sheet['M'][1:]]

amps_lente = [s.value for s in sheet['P'][1:]]

In [5]:
counter = 0
for path_onlypores,path_mask,path_rf,path_amp,path_onlypores_lente,path_mask_lente,path_rf_lente,path_amp_lente in zip(onlypores,masks,rfs,amps,onlypores_lentes,masks_lentes,rfs_lente,amps_lente):
    
    if (path_onlypores!= None) and (path_mask != None) and (path_rf != None) and (path_amp != None) and (datasets1[counter] == None) or (datasets2[counter] == None):

        try:

            print(f'Processing {ids[counter]}')
            
            print('Loading:',path_onlypores)

            onlypores = read_sequence(path_onlypores)
            print('Loading:',path_mask)
            mask = read_sequence(path_mask)
            print('Loading:',path_rf)
            rf = tiff.imread(path_rf)
            print('Loading:',path_amp)
            amp = tiff.imread(path_amp)

            output_folder = Path(path_onlypores).parent.parent / 'Datasets'

            if output_folder.exists() == False:
                output_folder.mkdir()

            datasetmaker.main(onlypores,mask,rf,amp,output_folder)

            sheet[f'I{counter+2}'] = str(output_folder / 'patch_vs_volfrac.csv')
            sheet[f'J{counter+2}'] = str(output_folder / 'samplepatch_vs_patch_mask.csv')
        
        except Exception as e:
            print(f'Error processing {ids[counter]}')
            print(e)
    
    if (path_onlypores_lente!= None) and (path_mask_lente != None) and (path_rf_lente != None) and (path_amp_lente != None) and (datasets1_lente[counter] == None) or (datasets2_lente[counter] == None):

        try:

            print(f'Processing {ids[counter]}')
            
            print('Loading:',path_onlypores_lente)

            onlypores = read_sequence(path_onlypores_lente)
            print('Loading:',path_mask_lente)
            mask = read_sequence(path_mask_lente)
            print('Loading:',path_rf_lente)
            rf = tiff.imread(path_rf_lente)
            print('Loading:',path_amp_lente)
            amp = tiff.imread(path_amp_lente)

            output_folder = Path(path_onlypores_lente).parent.parent / 'Datasets'

            if output_folder.exists() == False:
                output_folder.mkdir()

            datasetmaker.main(onlypores,mask,rf,amp,output_folder)

            sheet[f'Q{counter+2}'] = str(output_folder / 'patch_vs_volfrac.csv')
            sheet[f'R{counter+2}'] = str(output_folder / 'samplepatch_vs_patch_mask.csv')
        
        except Exception as e:
            print(f'Error processing {ids[counter]}')
            print(e)

    counter+=1

Processing 1_26
Loading: None
Error processing 1_26
no files found
Processing 1_26
Loading: \\192.168.10.254\NAS2_vol2_Main\11_AIRBUS\03_LabXCT\Panel Pegaso\Probetas\1\26\segmentation\onlypores_lente


Progress:  21%|██        | 72/350 [00:08<00:31,  8.87it/s]


KeyboardInterrupt: 

In [ ]:
wb.save(r'Y:\Database\Database.xlsx')